# Love Geometry

## Application consists of several modules:
- `peg_parser`
- `server`

Each of the modules has separate `consts.py` file with config valiables.
Please configure them up to your choice before starting the application server.

- `per_parser` has such default config variables:
    ```
    LOVE_CASE_SEPARATORS = [",", "but", "while", "and", "\n"]
    FEELINGS = ["loves", "hates"]
    SENTENCE_SEPARATOR = "."
    DOUBLE_ENDED_RELATIONSHIP_FLAG_KEYWORD = "mutually"
    ALLOW_HIGH_SELF_ESTEEM = False
    ```
    
PEG parser can work with 2 or more feelings.

PEG parser can work with 1 or more separator.

- `server` has such default config variables:
    ```
    VALIDATE_LOVE_STORIES = True
    VISUALIZE_CIRCLES_OF_AFFECTION = False
    ```
    
If `ALLOW_HIGH_SELF_ESTEEM` is `False` and `VALIDATE_LOVE_STORIES` is `True`, than cases like 'A loves A' will appear in errors.

`VISUALIZE_CIRCLES_OF_AFFECTION` flag will attempt to show visualization of "Circles of affection" on a server side.

In [63]:
import json
import requests

API_URL = "http://127.0.0.1:5000/"
PARSER_ENDPOINT = "parse-love-story"
CIRCLES_OF_AFFECTION_URL = "find-circles-of-affection"

def show_result(response):
    sentence_number = 1
    for i, item in enumerate(response):
        print(f"Sentence number: {sentence_number}")
        sentence_number += 1
        print(json.dumps(item, indent=4))

## Some cases to check Parser:

In [64]:
love_story = {"love_story": "A loves B."}
show_result(requests.post(API_URL + PARSER_ENDPOINT, json=love_story).json())

Sentence number: 1
{
    "errors": [],
    "data": {
        "A": {
            "loves": [
                "B"
            ]
        }
    }
}


In [65]:
love_story = {"love_story": "A loves B but B hates A. \n A hates B, A loves D while B loves C and D hates A."}
show_result(requests.post(API_URL + PARSER_ENDPOINT, json=love_story).json())

Sentence number: 1
{
    "errors": [],
    "data": {
        "A": {
            "loves": [
                "B"
            ]
        },
        "B": {
            "hates": [
                "A"
            ]
        }
    }
}
Sentence number: 2
{
    "errors": [],
    "data": {
        "A": {
            "hates": [
                "B"
            ],
            "loves": [
                "D"
            ]
        },
        "B": {
            "loves": [
                "C"
            ]
        },
        "D": {
            "hates": [
                "A"
            ]
        }
    }
}


#### The most common inconsistencies are checked with parser level validation. 
Example cases that are validated:

    - "A loves B and A loves B."
    - "A loves B and B loves C. A loves B and B loves C."
    - "A loves A" in case if configured

In [73]:
love_story = {"love_story": "A loves B and A loves B."}
result  =requests.post(API_URL + PARSER_ENDPOINT, json=love_story).json()
show_result(result)

Sentence number: 1
{
    "errors": [
        "Duplicated love case: `A loves B`"
    ],
    "data": {
        "A": {
            "loves": [
                "B",
                "B"
            ]
        }
    }
}


#### Serializer validates only duplicated sentences in story and raises `422 Unprocessable Entity` error if some.

In [74]:
love_story = {"love_story": "A loves B and B loves C. A loves B and B loves C."}
result  =requests.post(API_URL + PARSER_ENDPOINT, json=love_story)
print(result.text)


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>422 Unprocessable Entity</title>
<h1>Unprocessable Entity</h1>
<p>Love story has duplicated sentences</p>



#### Parser can recognize `mutually` keyword

In [75]:
love_story = {"love_story": "A mutually loves B."}
result  =requests.post(API_URL + PARSER_ENDPOINT, json=love_story).json()
show_result(result)

Sentence number: 1
{
    "errors": [],
    "data": {
        "A": {
            "loves": [
                "B"
            ]
        },
        "B": {
            "loves": [
                "A"
            ]
        }
    }
}


#### One person can love\hate more than one other person

In [76]:
love_story = {"love_story": "A loves B and A loves C and A hates D, A hates C."}
result  =requests.post(API_URL + PARSER_ENDPOINT, json=love_story).json()
show_result(result)

Sentence number: 1
{
    "errors": [],
    "data": {
        "A": {
            "hates": [
                "D",
                "C"
            ],
            "loves": [
                "B",
                "C"
            ]
        }
    }
}


#### But no duplicated feelings are allowed

In [77]:
love_story = {"love_story": "A mutually loves B and A loves C and A hates D, A hates C and B hates A."}
result  =requests.post(API_URL + PARSER_ENDPOINT, json=love_story).json()
show_result(result)

Sentence number: 1
{
    "errors": [
        "Duplicated love case: `B loves / hates A`"
    ],
    "data": {
        "A": {
            "hates": [
                "D",
                "C"
            ],
            "loves": [
                "B",
                "C"
            ]
        },
        "B": {
            "hates": [
                "A"
            ],
            "loves": [
                "A"
            ]
        }
    }
}


## Some cases to find Circles of Affection and cheaters:

In [78]:
love_story = {"love_story": "A loves B and B loves C and C loves D, D loves A and Q hates W and W hates E and E hates Q."}
result  =requests.post(API_URL + CIRCLES_OF_AFFECTION_URL, json=love_story).json()
print(json.dumps(result))


{"circles_of_affection": [{"loves": [["A", "B"], ["B", "C"], ["C", "D"], ["D", "A"]], "hates": [["Q", "W"], ["W", "E"], ["E", "Q"]]}], "cheaters": [[]]}


#### Someone considered as a cheater in case, if has more than one feeling to the same person.
* In case if cheater is found it's got removed from the circle of affection candidates and circle search happens without cheater.

In [79]:
love_story = {"love_story": "A loves B and B loves C and C loves D, D loves A and D hates A and C loves E and E loves A."}
result  =requests.post(API_URL + CIRCLES_OF_AFFECTION_URL, json=love_story).json()
print(json.dumps(result))

{"circles_of_affection": [{"loves": [["A", "B"], ["B", "C"], ["C", "E"], ["E", "A"]]}], "cheaters": [["D"]]}


In [80]:
print("Thank you.")

Thank you.
